##import library

In [58]:
import numpy as np
import pandas as pd
import imageio
import os
from skimage import io
from IPython.display import Image
from PIL import Image
import cv2
from scipy.signal import correlate2d
import matplotlib.pyplot as plt

##resize

In [59]:
from PIL import Image
import os
 # Replace with the path to your input folder
input_folders=[r"H:\GIT project\Machine Learning with Abolgasemi\Project\fake", r"H:\GIT project\Machine Learning with Abolgasemi\Project\real"]
output_folder = r'H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized'  # Replace with the path to your output folder
desired_size = (1024, 1024)

for input_folder in input_folders:
    for filename in os.listdir(input_folder):
        try:
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                with Image.open(os.path.join(input_folder, filename)) as image:
                # Convert palette (P) mode image to RGB or grayscale
                    if image.mode == "P"  or image.mode == "RGBA":
                         image = image.convert("RGB")  # or "L" for grayscale mode
                # Resize the image while maintaining aspect ratio
                    new_image = image.resize(desired_size)
                    output_path = os.path.join(output_folder, filename)
                    new_image.save(output_path)
                # Save the resized image to the output folder
                # print(f"Resized and saved: {output_path}")
        except (OSError, IOError) as e:
                print(f"Skipped invalid file: {filename} - {e}")



Skipped invalid file: 810199515_real_none_jungle_1.jpeg - cannot identify image file 'H:\\GIT project\\Machine Learning with Abolgasemi\\Project\\real\\810199515_real_none_jungle_1.jpeg'
Skipped invalid file: 810199515_real_none_jungle_10.jpeg - cannot identify image file 'H:\\GIT project\\Machine Learning with Abolgasemi\\Project\\real\\810199515_real_none_jungle_10.jpeg'


###print  image resolation to check

In [60]:
from PIL import Image
import os

folder_path = r'H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized'  # Replace with the path to your folder

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Check if the file is an image
    if os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        try:
            # Open the image and get its resolution
            with Image.open(file_path) as img:
                resolution = img.size
                print(f"Image: {filename}, Resolution: {resolution[0]}x{resolution[1]} pixels")
        except (IOError, SyntaxError) as e:
            print(f"Error processing {filename}: {str(e)}")
    else:
        print(f"Skipping {filename}: Not an image file")


Image: 150400011_fake_dall.e_jungle_1.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_jungle_2.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_jungle_3.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_jungle_4.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_jungle_5.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_mountain_1.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_mountain_2.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_mountain_3.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_mountain_4.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_mountain_5.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_sea_1.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_sea_2.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_sea_3.jpg, Resolution: 1024x1024 pixels
Image: 150400011_fake_dall.e_sea_4.jpg, Resolution: 1024x1024 p

##get saturation & hue & Value

In [61]:
def get_saturation_hue_Value(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Convert the image from BGR to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Split the HSV image into individual channels
    h, s, v = cv2.split(hsv_image)

    # Calculate the average saturation
    saturation = np.mean(s)
    hue = np.mean(h)
    value =  np.mean(v)
    return saturation , hue , value

#features

##Quantization Tables

In [62]:
from PIL import Image
import os

def extract_quantization_tables(image):
    # Open the image
   
    try:
        # Check if the image has quantization tables
        if not hasattr(image, 'quantization'):
            print(f"The image  does not contain quantization tables.")
            return [None , None , None , None]
    
        # Get the quantization tables
        quantization_tables = image.quantization
    
        # Check if the required quantization tables exist
        if len(quantization_tables) < 2:
            print(f"The image  does not have the required quantization tables.")
            return [None , None , None , None]
    
        # Extract Quantization Table 0 and Quantization Table 1
        quantization_table_0 = quantization_tables[0]
        quantization_table_1 = quantization_tables[1]
        # Calculate the mean along axis 0

        mean_table_0 = np.mean(quantization_table_0, axis=0)
        # Calculate the standard deviation along axis 0
        std_table_0 = np.std(quantization_table_0, axis=0)
        # Calculate the mean along axis 0
        mean_table_1 = np.mean(quantization_table_1, axis=0)
        # Calculate the standard deviation along axis 0
        std_table_1 = np.std(quantization_table_1, axis=0)
        # Return the extracted tables
    except:
        mean_table_0 ,std_table_0 , mean_table_1,std_table_1 = [None , None , None , None]
    return mean_table_0, std_table_0,mean_table_1,std_table_1

##jpeg quality

In [63]:
def get_jpeg_quality(image_path):
    with Image.open(image_path) as img:
        exif = img.info.get('exif')
        if exif is not None and exif[0:6] == b'Exif\x00\x00':
            # Check if EXIF data exists
            for i in range(4, len(exif), 12):
                if exif[i:i+2] == b'\xff\xc0':
                    # Found the APP0 marker for the quantization table
                    return exif[i+5]
        return None


#Watermark  detector total

###sub watermark1

In [64]:
from PIL import Image

def get_mean_color_watermark1_yellow(image_path, x_start=944, y_start=1009 , x_end= 958 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 256 and  mean_r > 240) and (mean_g < 256 and  mean_g > 240) and (mean_b < 115 and  mean_b > 95):
        return True
    else:
        return False


def get_mean_color_watermark1_lightblue(image_path, x_start=962, y_start=1009 , x_end= 972 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 75 and  mean_r > 58) and (mean_g <= 256 and  mean_g > 240) and (mean_b < 256 and  mean_b > 240):
        return True
    else:
        return False


def get_mean_color_watermark1_green(image_path, x_start=977, y_start=1009 , x_end= 989 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 89 and  mean_r > 71) and (mean_g <= 226 and  mean_g > 208) and (mean_b < 89 and  mean_b > 68):
        return True
    else:
        return False


def get_mean_color_watermark1_red(image_path, x_start=993, y_start=1009 , x_end= 1006 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 256 and  mean_r > 240) and (mean_g <= 120 and  mean_g > 100) and (mean_b < 70 and  mean_b > 50):
        return True
    else:
        return False


def get_mean_color_watermark1_darkblue(image_path, x_start=1009, y_start=1009 , x_end= 1023 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 70 and  mean_r > 50) and (mean_g <= 80 and  mean_g > 60) and (mean_b < 256 and  mean_b > 240):
        return True
    else:
        return False




##sub watermark2

In [66]:
image_path = r"H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized\810199567_fake_dalle_mountain_2.jpeg"

def get_mean_color_watermark2_5th_layer(image_path, x_start=0, y_start=974 , x_end= 4 ,  y_end= 981):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 203 and  mean_r > 175) and (mean_g <= 200 and  mean_g > 175) and (mean_b < 205 and  mean_b > 175):
        return True
    else:
        return False


def get_mean_color_watermark2_4th_layer(image_path, x_start=0, y_start=984 , x_end= 4 ,  y_end= 990):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 155 and  mean_r > 130) and (mean_g <= 155 and  mean_g > 130) and (mean_b < 165 and  mean_b > 130):
        return True
    else:
        return False


def get_mean_color_watermark2_3th_layer(image_path, x_start=0, y_start=995 , x_end= 4 ,  y_end= 1001):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 110 and  mean_r > 90) and (mean_g <= 110 and  mean_g > 90) and (mean_b < 115 and  mean_b > 95):
        return True
    else:
        return False


def get_mean_color_watermark2_2th_layer(image_path, x_start=0, y_start=1005 , x_end= 4 ,  y_end= 1012):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 95 and  mean_r > 65) and (mean_g <= 95 and  mean_g > 70) and (mean_b < 95 and  mean_b > 70):
        return True
    else:
        return False



def get_mean_color_watermark2_1th_layer(image_path, x_start=0, y_start=1015 , x_end= 4 ,  y_end= 1023):
    image = Image.open(image_path)
    cropped_image = image.crop((x_start, y_start, x_end, y_end))
    pixels = cropped_image.getdata()

    # Calculate the mean RGB values
    total_r, total_g, total_b = 0, 0, 0
    pixel_count = 0

    for r, g, b in pixels:
        total_r += r
        total_g += g
        total_b += b
        pixel_count += 1

    mean_r = int(total_r / pixel_count)
    mean_g = int(total_g / pixel_count)
    mean_b = int(total_b / pixel_count)
    if (mean_r < 45 and  mean_r > 22) and (mean_g <= 45 and  mean_g > 22) and (mean_b < 47 and  mean_b > 22):
        return True
    else:
        return False

mean_color = get_mean_color_watermark2_1th_layer(image_path)
print("Mean color:", mean_color)



Mean color: True


##watermark1

In [67]:
# Example usage
image_path = r"H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized\810199584_fake_dall.e_mountain_1.jpeg"
def Has_watermark1(image_path):
    if (get_mean_color_watermark1_yellow(image_path) and get_mean_color_watermark1_lightblue(image_path) and get_mean_color_watermark1_green(image_path) and
        get_mean_color_watermark1_red(image_path)  and get_mean_color_watermark1_darkblue(image_path)):
        return True
    else:
        return False

mean_color = Has_watermark1(image_path)
print("Mean color:", mean_color)

Mean color: True


##Watermark 2 detector

In [68]:
# Example usage
image_path = r"H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized\830401079_real_none_forest_7.jpeg.jpg"
def Has_watermark2(image_path):
    if (get_mean_color_watermark2_5th_layer(image_path) and get_mean_color_watermark2_4th_layer(image_path) and get_mean_color_watermark2_3th_layer(image_path) and
        get_mean_color_watermark2_2th_layer(image_path)  and get_mean_color_watermark2_1th_layer(image_path)):
        return True
    else:
        return False

mean_color = Has_watermark2(image_path)
print("Mean color:", mean_color)

Mean color: False


##Watermark  detector total main func

In [69]:
image_path = r"H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized\810199567_fake_dalle_sea_1.jpeg"

def Has_watermark(image_path):
    if (Has_watermark2(image_path) or Has_watermark1(image_path)):
        return True
    else:
        return False
mean_color = Has_watermark(image_path)
print("Mean color:", mean_color)

Mean color: True


#correct where made

In [89]:
def correct_where_made(y_where_made_uncleaned):
    unique_names = set(y_where_made_uncleaned)
    print("diffrent where made (with out cleaning):")
    for name in unique_names:
        print(name)
    y_where_made_cleaned = y_where_made_uncleaned
    for i in range(len(y_where_made_cleaned)):
        if (y_where_made_cleaned[i] == "DALLE"  or  y_where_made_cleaned[i] == "dall.e" or y_where_made_cleaned[i] == "dalle"
          or  y_where_made_cleaned[i] == "delle" or y_where_made_cleaned[i] == "dall" or  y_where_made_cleaned[i] == "dalleminbot" or y_where_made_cleaned[i] == "dallemini"
          or  y_where_made_cleaned[i] == "DallE" or y_where_made_cleaned[i] == "dalleminibot"
          or  y_where_made_cleaned[i] == "bing" or y_where_made_cleaned[i] == "dalleminibot"
          or  y_where_made_cleaned[i] == "DallE" or y_where_made_cleaned[i] == "dalleminibot"):
            y_where_made_cleaned[i]= "DALL.E"
        if y_where_made_cleaned[i] == "none":
            y_where_made_cleaned[i]= "None"
        if y_where_made_cleaned[i] == "stable" :
         y_where_made_cleaned[i]= "Stable"
        if y_where_made_cleaned[i] == "dreamstudio" or y_where_made_cleaned[i] == "dream" :
         y_where_made_cleaned[i]= "dreamai"

    unique_names2 = set(y_where_made_cleaned)
    print("diffrent where made  (after cleaning) :")
    for name in unique_names2:
        print(name)
    return y_where_made_cleaned

#correct category

In [71]:
def correct_category(y_category_uncleaned):
      unique_names = set(y_category_uncleaned)
      print("diffrent category (with out cleaning):")
      for name in unique_names:
          print(name)
      y_category_cleaned = y_category_uncleaned
      for i in range(len(y_category_cleaned)):
          if y_category_cleaned[i] == "mountain":
              y_category_cleaned[i]= "Mountain"
          if y_category_cleaned[i] == "see" or  y_category_cleaned[i] == "sea" or y_category_cleaned[i] == "sea5.jpeg":
              y_category_cleaned[i]= "Sea"
          if y_category_cleaned[i] == "junlge" or  y_category_cleaned[i] == "jungle" or  y_category_cleaned[i] == "forest":
           y_category_cleaned[i]= "Jungle"

      unique_names = set(y_category_cleaned)
      print("diffrent  category (after cleaning) :")
      for name in unique_names:
          print(name)
      return y_category_cleaned

#correct real or fake

In [72]:
def correct_real_or_fake(y_real_fake_uncleaned):
    unique_names = set(y_real_fake_uncleaned)
    print("fake or real (with out cleaning):")
    for name in unique_names:
        print(name)
    y_real_fake_cleaned = y_real_fake_uncleaned
    for i in range(len(y_real_fake_cleaned)):
        if y_real_fake_cleaned[i] == "fake":
            y_real_fake_cleaned[i]= "Fake"
        if y_real_fake_cleaned[i] == "real":
            y_real_fake_cleaned[i]= "Real"


    unique_names = set(y_real_fake_cleaned)
    print("diffrent FAKE OR REAL (after cleaning) :")
    for name in unique_names:
        print(name)
    return y_real_fake_cleaned

#save file in CSV

In [73]:
import pandas as pd

def save_lists_to_csv(lists, filename,columns_given):
    transposed_lists = list(map(list, zip(*lists)))  # Transpose the lists
    df = pd.DataFrame(transposed_lists, columns=columns_given)
    df.to_csv(filename, index=False)
    print(f"Lists saved to '{filename}' successfully.")


#luminance gradient

In [44]:

def luminance_gradient(image):
        # Calculate the gradient using Sobel operator
        gradient_x_sobel = cv2.Sobel(image, cv2.CV_64F, 1, 0)
        gradient_y_sobel = cv2.Sobel(image, cv2.CV_64F, 0, 1)
        gradient_x_scharr = cv2.Scharr(image, cv2.CV_64F, 1, 0)
        gradient_y_scharr = cv2.Scharr(image, cv2.CV_64F, 0, 1)

        # Calculate the magnitude and direction of the gradient
        gradient_magnitude_scharr = np.sqrt(gradient_x_scharr ** 2 + gradient_y_scharr ** 2)
        gradient_magnitude_sobel = np.sqrt(gradient_x_sobel ** 2 + gradient_y_sobel ** 2)

        # Extract features from the gradient magnitude
        features_scharr = []
        features_scharr.append(np.mean(gradient_magnitude_scharr))
        features_scharr.append(np.std(gradient_magnitude_scharr))
        features_scharr.append(skew(gradient_magnitude_scharr.flatten()))

        features_sobel = []
        features_sobel.append(np.mean(gradient_magnitude_sobel))
        features_sobel.append(np.std(gradient_magnitude_sobel))
        features_sobel.append(skew(gradient_magnitude_sobel.flatten()))
        return features_scharr , features_sobel



#local binary pattern

In [74]:
import numpy as np
from skimage.feature import local_binary_pattern
from skimage import io, color

# Load the image
folder_path = r'E:\University\Term6\ML\Spring 401\Final Project\ML_project\All_in__One_resized'


def extract_lbp_features(image_path, radius=1, n_points=8):
    # Load the image
    image = io.imread(image_path)



    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)
    gray_image = np.clip(gray_image * 255, 0, 255).astype(np.uint8)

    # Calculate LBP
    lbp_image = local_binary_pattern(gray_image, n_points * radius, radius, method='uniform')

    # Calculate histogram of LBP patterns
    histogram, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))

    # Normalize the histogram
    normalized_histogram = histogram.astype(np.float32) / np.sum(histogram)

    return normalized_histogram





#glcms

In [75]:
def extract_glcms(image):
    # Convert the image to grayscale if it's in color

    image = color.rgb2gray(image)

    # Compute GLCM matrix
    image_unit = np.clip(image * 255, 0, 255).astype(np.uint8)
    glcm = feature.graycomatrix(image_unit, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)

    # Calculate GLCM features
    contrast = feature.graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = feature.graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = feature.graycoprops(glcm, 'homogeneity')[0, 0]
    energy = feature.graycoprops(glcm, 'energy')[0, 0]
    correlation = feature.graycoprops(glcm, 'correlation')[0, 0]

    return [contrast, dissimilarity, homogeneity, energy, correlation]




#HOG

In [76]:
import numpy as np
import os
from skimage import io, feature

# Function to extract HOG features from an image
def extract_hog_features(image):
    # Convert the image to grayscale
    image_gray = color.rgb2gray(image)

    # Calculate HOG features
    hog_features = feature.hog(image_gray, orientations=9, pixels_per_cell=(8, 8),
                               cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)

    return hog_features

#noise

In [78]:
import cv2
import numpy as np

def extract_noise_features(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Calculate the noise features
    mean_value = np.mean(image)
    std_deviation = np.std(image)

    return mean_value , std_deviation




#Fourier

In [79]:
import cv2
import numpy as np

def extract_fourier_features(img):
    # Check the number of channels in the input image
    if len(img.shape) == 3 and img.shape[-1] == 3:
        # Convert the input image to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        # The input image is already grayscale or has only one channel
        gray_img = img

    # Apply the Fourier Transform to the grayscale image
    try:
        f = np.fft.fft2(gray_img)
        fshift = np.fft.fftshift(f)

        # Compute the magnitude spectrum of the Fourier Transform
        magnitude_spectrum = 20 * np.log(np.abs(fshift))

        # Extract the mean, variance, skewness, and kurtosis of the magnitude spectrum
        mean_spectrum = np.mean(magnitude_spectrum)
        var_spectrum = np.var(magnitude_spectrum)
        skew_spectrum = np.mean((magnitude_spectrum - mean_spectrum) ** 3) / var_spectrum ** (3/2)
        kurt_spectrum = np.mean((magnitude_spectrum - mean_spectrum) ** 4) / var_spectrum ** 2 - 3

        # Return the Fourier Transform features as a list
        features = [mean_spectrum, var_spectrum, skew_spectrum, kurt_spectrum]

    except:
        # If an exception occurs, return None as the features
        features =[None, None, None, None]

    return features

#ELA

In [80]:
import cv2
import numpy as np
from PIL import Image, ImageChops

def ela(image_path, quality=90):
    # Load the image
    image = Image.open(image_path)

    # Save the image with a temporary filename and the specified quality
    temp_path = "temp.jpg"
    image.save(temp_path, "JPEG", quality=quality)

    # Load the ELA image
    ela_image = Image.open(temp_path)

    # Calculate the absolute difference between the original and ELA image
    diff_image = ImageChops.difference(image, ela_image)

    # Calculate the grayscale histogram of the difference image
    histogram = diff_image.histogram()

    # Calculate the ELA features
    max_value = max(histogram)
    min_value = min(histogram)
    mean_value = np.mean(histogram)
    std_deviation = np.std(histogram)

    # Remove the temporary ELA image file
    ela_image.close()
    diff_image.close()
    image.close()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    features = [max_value, min_value, mean_value, std_deviation]

    return features


#wavelet

In [81]:
import cv2
import numpy as np
import pywt

def extract_wavelet_features(img):
    # Check the number of channels in the input image
        # Convert the input image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply the wavelet transform to the grayscale image
    try:
        coeffs = pywt.dwt2(gray_img, 'haar')
        cA, (cH, cV, cD) = coeffs

        # Extract the mean, variance, skewness, and kurtosis of each coefficient
        mean_cA = np.mean(cA)
        var_cA = np.var(cA)
        skew_cA = np.mean((cA - mean_cA) ** 3) / var_cA ** (3/2)
        kurt_cA = np.mean((cA - mean_cA) ** 4) / var_cA ** 2 - 3

        mean_cH = np.mean(cH)
        var_cH = np.var(cH)
        skew_cH = np.mean((cH - mean_cH) ** 3) / var_cH ** (3/2)
        kurt_cH = np.mean((cH - mean_cH) ** 4) / var_cH ** 2 - 3

        mean_cV = np.mean(cV)
        var_cV = np.var(cV)
        skew_cV = np.mean((cV - mean_cV) ** 3) / var_cV ** (3/2)
        kurt_cV = np.mean((cV - mean_cV) ** 4) / var_cV ** 2 - 3

        mean_cD = np.mean(cD)
        var_cD = np.var(cD)
        skew_cD = np.mean((cD - mean_cD) ** 3) / var_cD ** (3/2)
        kurt_cD = np.mean((cD - mean_cD) ** 4) / var_cD ** 2 - 3

        # Return the wavelet transform features as a list
        features = [mean_cA, var_cA, skew_cA, kurt_cA,
                    mean_cH, var_cH, skew_cH, kurt_cH,
                    mean_cV, var_cV, skew_cV, kurt_cV,
                    mean_cD, var_cD, skew_cD, kurt_cD]

    except:
        # If an exception occurs, return None as the features
        features = [None, None, None, None,
                    None, None, None, None,
                    None, None, None, None,
                    None, None, None, None]

    return features



#read photo and extract features

In [82]:
from PIL import Image
from scipy.stats import skew
# Load the images from the folder
folder_path_1= r'H:\GIT project\Machine Learning with Abolgasemi\Project\All_in__One_resized'
folder_path_2= r'H:\GIT project\Machine Learning with Abolgasemi\Project\All_in_One'

image_files = os.listdir(folder_path_1)
# Features
mean_color_red = []
mean_color_blue = []
mean_color_green = []
y_category_uncleaned=[]
y_real_fake_uncleaned = []
y_where_made_uncleaned = []
mean_quant_table_0_list= []
mean_quant_table_1_list= []
std_quant_table_0_list= []
std_quant_table_1_list= []
jpeg_quality=[]
watermark =[]
saturation_list = []
hue_list = []
Value_list = []
mean_scharr = []
std_scharr = []
skew_scharr = []
mean_sobel = []
std_sobel = []
skew_sobel = []
mean_hog_list = []
std_hog_list = []
mean_noise_list = []
std_noise_list = []
contrast_glcms_list = []
dissimilarity_glcms_list = []
homogeneity_glcms_list = []
energy_glcms_list = []
correlation_glcms_list = []
LBP1 = []
LBP2= []
LBP3 = []
LBP4 = []
LBP5 = []
LBP6 = []
LBP7 = []
LBP8 = []
LBP9 = []
LBP10 = []
image_name = []
mean_spectrum_list = []
var_spectrum_list = []
skew_spectrum_list = []
kurt_spectrum_list = []
max_value_list_ela = []
min_value_list_ela = []
mean_value_list_ela= []
std_deviation_list_ela = []
mean_cA_wavelet = []
var_cA_wavelet = []
skew_cA_wavelet = []
kurt_cA_wavelet = []
mean_cH_wavelet = []
var_cH_wavelet = []
skew_cH_wavelet = []
kurt_cH_wavelet = []
mean_cV_wavelet = []
var_cV_wavelet = []
skew_cV_wavelet = []
kurt_cV_wavelet = []
mean_cD_wavelet = []
var_cD_wavelet = []
skew_cD_wavelet = []
kurt_cD_wavelet = []
student_num = []




# Classify each image
for file in image_files:
    try:
        image_path = os.path.join(folder_path_1, file)
        image1 = io.imread(image_path)
        image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
        image_path2 = os.path.join(folder_path_2, file)
        image2 = Image.open(image_path2)
        
        mean_table_0, std_table_0,mean_table_1,std_table_1 = extract_quantization_tables(image2)
        mean_quant_table_0_list.append(mean_table_0)
        mean_quant_table_1_list.append(mean_table_1)
        std_quant_table_0_list.append(std_table_0)
        std_quant_table_1_list.append(std_table_1)

        features_scharr ,features_sobel  =luminance_gradient(image1)
        mean_scharr.append(features_scharr[0])
        std_scharr.append(features_scharr[1])
        skew_scharr.append(features_scharr[2])
        mean_sobel.append(features_sobel[0])
        std_sobel.append(features_sobel[1])
        skew_sobel.append(features_sobel[2])

        image_name.append(file)

        LBP = extract_lbp_features(image_path)
        LBP1.append(LBP[0])
        LBP2.append(LBP[1])
        LBP3 .append(LBP[2])
        LBP4 .append(LBP[3])
        LBP5 .append(LBP[4])
        LBP6 .append(LBP[5])
        LBP7 .append(LBP[6])
        LBP8 .append(LBP[7])
        LBP9 .append(LBP[8])
        LBP10 .append(LBP[9])

        contrast_glcms, dissimilarity_glcms, homogeneity_glcms, energy_glcms, correlation_glcms = extract_glcms(image1)
        contrast_glcms_list.append(contrast_glcms)
        dissimilarity_glcms_list.append(dissimilarity_glcms)
        homogeneity_glcms_list.append(homogeneity_glcms)
        energy_glcms_list.append(energy_glcms)
        correlation_glcms_list.append(correlation_glcms)

        HOG = extract_hog_features(image1)
        mean_hog_list.append(np.mean(np.array(HOG), axis=0))
        std_hog_list.append(np.std(np.array(HOG), axis=0))


        mean_noise  , std_noise = extract_noise_features(image_path)
        mean_noise_list.append(mean_noise)
        std_noise_list.append(std_noise)

        mean_spectrum, var_spectrum, skew_spectrum, kurt_spectrum = extract_fourier_features(image)
        mean_spectrum_list.append(mean_spectrum)
        var_spectrum_list.append(var_spectrum)
        skew_spectrum_list.append(skew_spectrum)
        kurt_spectrum_list.append(kurt_spectrum)


        max_value_ela, min_value_ela, mean_value_ela, std_deviation_ela = ela(image_path, quality=90)
        max_value_list_ela.append(max_value_ela)
        min_value_list_ela.append(min_value_ela)
        mean_value_list_ela.append(mean_value_ela)
        std_deviation_list_ela.append(std_deviation_ela)

        mean_cA, var_cA, skew_cA, kurt_cA, mean_cH, var_cH, skew_cH, kurt_cH, mean_cV, var_cV, skew_cV, kurt_cV, mean_cD, var_cD, skew_cD, kurt_cD = extract_wavelet_features(image1)
        mean_cA_wavelet.append(mean_cA)
        var_cA_wavelet.append(var_cA)
        skew_cA_wavelet.append(skew_cA)
        kurt_cA_wavelet.append(kurt_cA)
        mean_cH_wavelet.append(mean_cH)
        var_cH_wavelet.append(var_cH)
        skew_cH_wavelet.append(skew_cH)
        kurt_cH_wavelet.append(kurt_cH)
        mean_cV_wavelet.append(mean_cV)
        var_cV_wavelet.append(var_cV)
        skew_cV_wavelet.append(skew_cV)
        kurt_cV_wavelet.append(kurt_cV)
        mean_cD_wavelet.append(mean_cD)
        var_cD_wavelet.append(var_cD)
        skew_cD_wavelet.append(skew_cD)
        kurt_cD_wavelet.append(kurt_cD)

        mean_color_blue.append( np.mean(image[:,:,0]))
        mean_color_green.append( np.mean(image[:,:,1]))
        mean_color_red.append( np.mean(image[:,:,2]))

        watermark.append(Has_watermark(image_path))

        saturation, hue , Value = get_saturation_hue_Value(image_path)
        saturation_list.append(saturation)
        hue_list.append(hue)
        Value_list.append(Value)

        try:
            parts = image_path.split(folder_path_1)[1].split("_")
            y_real_fake_uncleaned.append(parts[1])
            y_where_made_uncleaned.append(parts[2])
            y_category_uncleaned.append( parts[3])
            student_num.append(parts[0][1:])
        except:
            parts = image_path.split(folder_path_1)[1].split("-")
            y_real_fake_uncleaned.append(parts[1])
            y_where_made_uncleaned.append(parts[2])
            y_category_uncleaned.append( parts[3])
            student_num.append(parts[0][1:])
    except (OSError, IOError) as e:
        print(f"Skipped invalid file: {image_path} - {e}")


y_where_made_cleaned = correct_where_made(y_where_made_uncleaned)
y_category_cleaned = correct_category(y_category_uncleaned)
y_real_fake_cleaned = correct_real_or_fake(y_real_fake_uncleaned) 

feature_lists = [mean_color_blue , mean_color_green, mean_color_red ,saturation_list ,hue_list, Value_list, watermark,
                 mean_scharr ,std_scharr , skew_scharr ,mean_sobel,std_sobel , skew_sobel ,mean_hog_list ,std_hog_list ,
                 mean_noise_list,std_noise_list , mean_spectrum_list , var_spectrum_list , skew_spectrum_list , kurt_spectrum_list,
                 contrast_glcms_list , dissimilarity_glcms_list , homogeneity_glcms_list , energy_glcms_list , correlation_glcms_list ,
                 LBP1,LBP2,LBP3,LBP4,LBP5,LBP6,LBP7,LBP8,LBP9,LBP10,
                 mean_cA_wavelet, var_cA_wavelet , skew_cA_wavelet , kurt_cA_wavelet , mean_cH_wavelet , var_cH_wavelet , skew_cH_wavelet , kurt_cH_wavelet,
                 mean_cV_wavelet,var_cV_wavelet,skew_cV_wavelet,kurt_cV_wavelet,mean_cD_wavelet,var_cD_wavelet,skew_cD_wavelet,kurt_cD_wavelet,
                 mean_quant_table_0_list , mean_quant_table_1_list , std_quant_table_0_list , std_quant_table_1_list,
                 max_value_list_ela , min_value_list_ela ,mean_value_list_ela  , std_deviation_list_ela,student_num,
                 y_where_made_cleaned , y_category_cleaned , y_real_fake_cleaned ,image_name
                 ]
columns_given = ["blue" , "green" , "red" , "saturation", "hue","Value" , "watermark" ,
                 "mean_scharr" ,"std_scharr" , "skew_scharr" , "mean_sobel" , "std_sobel" , "skew_sobel" ,"mean_hog_list" ,"std_hog_list" ,
                 "mean_noise_list" , "std_noise_list", "mean_spectrum_list" ,  "var_spectrum_list" , "skew_spectrum_list" , "kurt_spectrum_list" ,
                 "contrast_glcms_list" , "dissimilarity_glcms_list" , "homogeneity_glcms_list" , "energy_glcms_list " , "correlation_glcms_list " ,
                 "LBP1","LBP2","LBP3","LBP4","LBP5","LBP6","LBP7","LBP8","LBP9","LBP10",
                 "mean_cA_wavelet", "var_cA_wavelet" , "skew_cA_wavelet" , "kurt_cA_wavelet" , "mean_cH_wavelet" , "var_cH_wavelet" , "skew_cH_wavelet" , "kurt_cH_wavelet",
                 "mean_cV_wavelet","var_cV_wavelet","skew_cV_wavelet" ,"kurt_cV_wavelet","mean_cD_wavelet","var_cD_wavelet","skew_cD_wavelet","kurt_cD_wavelet",
                "mean_quant_table_0_list" , "mean_quant_table_1_list" , "std_quant_table_0_list" , "std_quant_table_1_list",
                "max_value_list_ela" , "min_value_list_ela" ,"mean_value_list_ela"  , "std_deviation_list_ela","student_num",
                "y_where_made_uncleaned" ,"y_category_uncleaned" ,  "y_real_fake_uncleaned"  , "image_name"
                 ]
filename = "output_file.csv"
save_lists_to_csv(feature_lists, filename,columns_given)


C:\Users\M.M\AppData\Local\Temp\ipykernel_21376\3843450264.py:19: RuntimeWarning: divide by zero encountered in log
  magnitude_spectrum = 20 * np.log(np.abs(fshift))
C:\Users\M.M\AppData\Roaming\Python\Python311\site-packages\numpy\core\_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\M.M\AppData\Local\Temp\ipykernel_21376\3843450264.py:24: RuntimeWarning: invalid value encountered in subtract
  skew_spectrum = np.mean((magnitude_spectrum - mean_spectrum) ** 3) / var_spectrum ** (3/2)
C:\Users\M.M\AppData\Local\Temp\ipykernel_21376\3843450264.py:25: RuntimeWarning: invalid value encountered in subtract
  kurt_spectrum = np.mean((magnitude_spectrum - mean_spectrum) ** 4) / var_spectrum ** 2 - 3


The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not contain quantization tables.
The image  does not 